In [1]:
# Local
from resources import Tools # type: ignore
from resources import FlowTools # type: ignore

# Math
import numpy as np
import matplotlib.pyplot as plt

# Data
import cv2 as cv

In [2]:
# Get image from file
file = 'Source/Vary Clouds (South Horizon)/#9 2024-04-08 13.05.13.png'
Img1, Img2 = np.hsplit(cv.imread(file)[:,:,::-1], 2)
imgH, imgW = Img1.shape[:2]

# General
Tools.displaySettings()
order = 2

In [ ]:
# Detect and match features
featureFinder = cv.SIFT_create()
kp1, des1 = featureFinder.detectAndCompute(cv.cvtColor(Img1[:,:,::-1], cv.COLOR_BGR2GRAY), None)
kp2, des2 = featureFinder.detectAndCompute(cv.cvtColor(Img2[:,:,::-1], cv.COLOR_BGR2GRAY), None)
matcher = cv.BFMatcher(crossCheck=True)
matches = matcher.match(des1, des2)

""" Filter with out matches lower than median similarity, then use RANSAC """
def RANSACMedSim(matches, t=30):
    matches = sorted(matches, key=lambda m: m.distance)[:len(matches)//2]
    p1, p2 = Tools.getMatchPoints(matches, kp1, kp2)
    _, labels = cv.findHomography(p1.T, p2.T, cv.RANSAC, t)
    return [matches[i] for i,l in enumerate(labels) if l == 1]

# Filter matches and determine alignment
filterMatches = RANSACMedSim(matches)
c1 = c2 = np.zeros((Tools.polyTerms2D(order), 2))
p1G, p2G, translation, rotation, center, _ = Tools.globalAlignment(*Tools.getMatchPoints(filterMatches, kp1, kp2))
_, _, c1, c2 = Tools.localAlignment(p1G, p2G, c1, c2, order, 'Powell', 1e-18)

# Apply alignment and blend
p0 = np.array([np.repeat(range(imgW), imgH), np.tile(range(imgH), imgW)])
panorama, _, _, p1, p2 = Tools.mergeAndWarpImages(p0, p0, c1, c2, order, translation, rotation, center, [Img1, Img2])
img1, img2 = np.hsplit(Tools.refineAndConcatenateOverlaps(p0, p1, p2, [Img1, Img2]), 2)

# Plot result from feature detection
_, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8), width_ratios=(1.4, 1), constrained_layout=True)
ax1.imshow(panorama/np.max(panorama)); ax1.set_title('Panorama after Feature Detection'); ax1.set_axis_off()
ax2.imshow((img1+img2)/np.max(img1+img2)); ax2.set_title('Sum of Refined Overlaps'); ax2.set_axis_off()

In [4]:
# Optical Flow
dic, _, _ = FlowTools.findBestGPC(img1/np.max(img1), img2/np.max(img2))
dic.rollWindowAndFindBestGPC(stepLength=150, windowSize=200, extraWiggle=50)
dic.filterMissMatched(ransac_tol=5)
dic.warp(order)
dic.optimizeWarping(order)
dic.warp(order=order, optimized=True)
c = dic.OptimizeResult.x.reshape(dic.sUV0.shape)
c1, c2 = c1+c, c2-c

In [ ]:
panorama, _, _, p1, p2 = Tools.mergeAndWarpImages(p0, p0, c1, c2, order, translation, rotation, center, [Img1, Img2])
o1O, o2O = np.hsplit(Tools.refineAndConcatenateOverlaps(p0, p1, p2, [Img1, Img2]), 2)
i1, i2 = FlowTools.padImages(img1+img2, o1O+o2O); diff = np.abs(i1-i2)

_, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8), constrained_layout=True)
ax1.imshow(np.abs(img1-img2)/np.max(np.abs(img1-img2))); ax1.set_title('Difference between Images after Feature Detection'); ax1.set_axis_off()
ax2.imshow(np.abs(o1O-o2O)/np.max(np.abs(o1O-o2O)));  ax2.set_title('Difference between Images after Optical Flow'); ax2.set_axis_off()
_, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8), width_ratios=(1.4, 1), constrained_layout=True)
ax1.imshow(panorama/np.max(panorama)); ax1.set_title('Panorama after Feature Detection and Optical Flow'); ax1.set_axis_off()
ax2.imshow(diff/np.max(diff)); ax2.set_title('Difference before and after Optical Flow '); ax2.set_axis_off()